In [23]:
#Note This is the receiver code which need to be implemented in one
#    desktop and then sender code need to be run in another device with proper host and port label which can 
#    secure the connection between the two devices and send data properly.
"""
    Protocol procedure at the sending end as described in USLP section 4.2 
    MAP Packet Service (MAP Packet Processing) and MAP Access Service (MAPA_SDU Generation)
    have been implemented here and the code given for sender. For more information on the services refer comment in sender code.

    """
import socket

def receive_data(host='localhost', port=12345):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")
            message_parts = []
            while True:
                header = conn.recv(2)
                if not header:
                    break 
                payload_length = int.from_bytes(header, byteorder='big') - 1  # QoS (Quality of Service) byte
                qos = int.from_bytes(conn.recv(1), byteorder='big')  # Read QoS byte
                payload = conn.recv(payload_length).decode('utf-8')
                message_parts.append(payload)
                print(f"Received packet with QoS {qos}: {payload}")
            message = ''.join(message_parts)
            print(f"Complete message: {message}")

if __name__ == "__main__":
    receive_data()


Connected by ('127.0.0.1', 62895)
Received packet with QoS 1: Hi. This message will be segmented into smaller 
Received packet with QoS 1: packets according to the MAP processing function
Received packet with QoS 1:  from USLP frame.
Complete message: Hi. This message will be segmented into smaller packets according to the MAP processing function from USLP frame.


In [24]:
"""
    USLP TRANSFER FRAME as described in USLP section 4.1
    A USLP Transfer Frame shall encompass the major fields, positioned contiguously, in the following sequence:
a) Transfer Frame Primary Header (4 to 14 octets, mandatory);
b) Transfer Frame Insert Zone (integral number of octets, optional);
c) Transfer Frame Data Field (integral number of octets, mandatory);
d) Operational Control Field (4 octets, optional); and
e) Frame Error Control Field (2 octets, optional).
Here I implemented part a,c,e in this code for simplicity and the frame is checked accordingly.
This is the receiver code which need to be implemented in one
    desktop and the sender code need to be run in another device with proper host and port label which can 
    secure the connection between the two devices and send data properly.
    """
import socket
import crcmod
import struct

# CRC-16-CCITT function
crc16_func = crcmod.predefined.mkPredefinedCrcFun('crc-ccitt-false')

def verify_frame(frame):
    # Extract the frame count, data, and FECF
    frame_count, = struct.unpack('>H', frame[:2])
    data = frame[2:-2]  # Exclude frame count and FECF
    received_crc, = struct.unpack('>H', frame[-2:])
    calculated_crc = crc16_func(frame[:-2])  # Calculate CRC over frame excluding FECF
    
    # Verify CRC
    if received_crc == calculated_crc:
        print(f"Frame {frame_count} verified successfully. Data: {data.decode('utf-8')}")
    else:
        print(f"Frame {frame_count} CRC check failed.")

def receive_uslp_frames(host='localhost', port=12345):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.bind((host, port))
        sock.listen()
        conn, addr = sock.accept()
        with conn:
            print('Connected by', addr)
            while True:
                # Assuming a blocking call to recv() for simplicity
                # A real-world scenario would need more robust frame extraction logic
                frame = conn.recv(1024)  # Adjust buffer size as needed
                if not frame:
                    break
                verify_frame(frame)
                

if __name__ == "__main__":
    receive_uslp_frames()


Connected by ('127.0.0.1', 62898)
Frame 0 verified successfully. Data:  This is a longer message that will be segmented into smaller packets according to the MAP processing function.
